In [8]:
import cv2
import numpy as np
from PIL import Image
def imread(path):
    img = cv2.imread(path).astype(np.float)
    if len(img.shape) == 2:
        # grayscale
        img = np.dstack((img,img,img))
    elif img.shape[2] == 4:
        # PNG with alpha channel
        img = img[:,:,:3]
    return img

def imsave(path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    Image.fromarray(img).save(path, quality=95)

In [9]:
import torch
from ultralytics import YOLO
import numpy as np

device: str = "mps" if torch.backends.mps.is_available() else "cpu"

model = YOLO('./nst/models/yolov8x-seg.pt')  # load a pretrained YOLOv8n segmentation model

<class 'ultralytics.nn.tasks.SegmentationModel'>


In [10]:
import cv2
imgcon = cv2.imread('content.png')
H, W, _ = imgcon.shape
results = model(imgcon)


0: 640x480 1 vase, 797.2ms
Speed: 9.1ms preprocess, 797.2ms inference, 16.0ms postprocess per image at shape (1, 3, 640, 480)


In [11]:
for result in results:
    for j, mask in enumerate(result.masks.data):
        mask = mask.cpu().numpy() * 255
        mask  =cv2.resize(mask, (W, H))
        cv2.imwrite('./mask.png', mask)
        break

In [12]:
imgtar = cv2.imread("target.png")
imgtar = cv2.cvtColor(imgtar, cv2.COLOR_BGR2RGB)
imgtar = imgtar.astype(np.uint8)
imgtar = cv2.resize(imgtar, (W, H))
mask = cv2.imread('./mask.png', cv2.IMREAD_GRAYSCALE)
_, binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
masked_overlay = cv2.bitwise_or(imgtar, imgtar, mask=binary_mask)
masked_overlay = masked_overlay.astype(np.uint8)
inverted_mask = cv2.bitwise_not(binary_mask.astype(np.uint8))
roi = cv2.bitwise_and(imgcon, imgcon, mask=inverted_mask)
result_image = cv2.add(roi, masked_overlay)
imsave('./final_output.png', result_image)